# Qiskit interoperability for QFT and VQE

We begin by loading all required packages. `QuantexQASM` wraps the OpenQASM syntax in a platform agnostic manner, allowing the generation of gate-calls in other QASM syntax formats, or alternatively to gate calls of other simulators, where applicable.

Here, we start with a demonstration of using QuantexQASM to run a state-vector simulation on Qiskit of the quantum Fourier transform. Note, that QuantexQASM when imported will begin by attempting to install qiskit into the given Python environment used by Julia's conda.

In [ ]:
using Pkg;
Pkg.activate("..")
using QuantexQASM

using PyCall
using Optim
using Plots

In [ ]:
qiskit = PyCall.pyimport("qiskit");
qiskit_execute = PyCall.pyimport("qiskit.execute");

In [ ]:
qubit_label_q = "qr"
qubit_label_c = "cr"
q_count = 3
c_count = q_count

qubit_indices = collect(0:q_count-1)

cct_qft = QuantexQASM.QFT.gen_qft(qubit_label_q, qubit_indices)

qasm_file = QuantexQASM.GateOps.gen_qasm_header(q_count, c_count, qubit_label_q, qubit_label_c )
qasm_file *= cct_qft
qasm_file *= QuantexQASM.GateOps.measure_qubits_all(qubit_indices, qubit_indices, qubit_label_q, qubit_label_c)

f = open("qft_test.qasm","w") 
for i in QuantexQASM.Utils.format_string_nl(qasm_file)
    write(f, i)
end
close(f) #

In [ ]:
circ = qiskit.QuantumCircuit.from_qasm_file("qft_test.qasm")
circ.draw()

In [ ]:
#Run the circuit
simulator = qiskit.Aer.get_backend("qasm_simulator")
job = qiskit_execute.execute(circ, simulator, shots=1000)
result = job.result()
counts = result.get_counts(circ)
print("\nTotal count for 00 and 11 are:",counts)

counts

Extract counts (v) and labels (k) from the above evaulation.

In [ ]:
k = [key for (key, val) in counts];
v = [val for (key, val) in counts];

In [ ]:
bar(k,v)

In [ ]:
lsrc = circ.draw("latex_source")
println(lsrc)

# VQE example: $\sigma_z$

Using the qiskit backend, we generate OpenQASM syntax to find the minimum eigenvalue of the Hamiltonian $H = \sigma_z$, which we already know as -1. We make use of the Optim.jl package internally, and can specify the number of calls required to evaluate the circuit using the given ansatz; in this instance the ansatz is $\vert \Psi \rangle = R_x(\theta)\vert 0 \rangle$, where $R_x = \exp\left(\frac{i\theta\sigma_x}{2}\right)$. 

In [ ]:
QuantexQASM.VQE.min_expec_val(1.0)

As an example, the ansatz is generated using the internal function `ansatz_gen`, accepting the qubit register, qubit index and theta value. It is possible to override this function to use an alternative formalism.

In [ ]:
QuantexQASM.VQE.ansatz_gen("qr",[0], 0.1)

Often the minisation algorithm may become stuck in a local minimima, and fail to return the correct expectation value. To aid with overcoming this, we can try a different solver algorithm, or to specify the required number of samples to ensure a more accurate expectation value.

In [ ]:
num_samples = 10000;
QuantexQASM.VQE.expec_val(Float64(pi)/4, num_samples)

To perform the minimisation we specify a function of accepting a signal argument and returning a sigmal value.

In [ ]:
g(x) = QuantexQASM.VQE.expec_val(x[1], num_samples)

Optimisation is performed as follows, where a good starting value and ansatz can ensure a good approximate eignenvalue.

In [ ]:
Optim.optimize(g, [pi/2], Optim.BFGS())